In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-reviews-dataset/test.tsv
/kaggle/input/movie-reviews-dataset/train.tsv


In [3]:
!pip install --upgrade numpy
!pip install --upgrade SciPy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/64/41/284783f1014685201e447ea976e85fed0e351f5debbaf3ee6d7645521f1d/numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dil

In [5]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import XLNetModel, XLNetTokenizer

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
#load data
train_df = pd.read_csv("/kaggle/input/movie-reviews-dataset/train.tsv", sep='\t')
test_df = pd.read_csv("/kaggle/input/movie-reviews-dataset/test.tsv", sep='\t')

# # merge the tables by Body ID
# train_df = pd.merge(df_body_train, df_stance_train, on='Body ID', how='inner')
# test_df = pd.merge(df_body_test, df_stance_test, on='Body ID', how='inner')

# null_counts_train = train_df.isnull().sum() #no nulls
# null_counts_test = test_df.isnull().sum()  #no nulls

total_rows_train = len(train_df)
total_rows_test = len(test_df)

unique_label_train = train_df['label'].unique()
unique_label_test = test_df['label'].unique()

print("TRAIN: Total number of rows: ",total_rows_train,", Unique labels:",unique_label_train)
print("TEST: Total number of rows: ",total_rows_test,", Unique labels:",unique_label_test)

# removing "n//a" rows and reprinting

train_df = train_df[train_df['label'] != 'n\\a']

total_rows_train = len(train_df)
total_rows_test = len(test_df)

unique_label_train = train_df['label'].unique()
unique_label_test = test_df['label'].unique()

print("TRAIN: Total number of rows: ",total_rows_train,", Unique labels:",unique_label_train)
print("TEST: Total number of rows: ",total_rows_test,", Unique labels:",unique_label_test)

# print(train_df.head())
# print(test_df.head())

# convert the last column i.e. the categorical column to a one hot encoded list. 
train_df['label'] = pd.get_dummies(train_df['label'],columns=train_df.columns).astype(int).values.tolist()
new_df_train = train_df[['lang','review', 'label']].copy()
# Passing colums as train.columns so that the encoding is consistent among train and test
test_df['label'] = pd.get_dummies(test_df['label'],columns=train_df.columns).astype(int).values.tolist()
new_df_test = test_df[['lang','review', 'label']].copy()

# train_df['label'] = train_df['label'].apply(lambda x: 1 if x == 'pos' else 0 if x == 'neg' else None)
# new_df_train = train_df[['lang','review', 'label']].copy()

# test_df['label'] = test_df['label'].apply(lambda x: 1 if x == 'pos' else 0 if x == 'neg' else None)
# new_df_test = test_df[['lang','review', 'label']].copy()



TRAIN: Total number of rows:  79852 , Unique labels: ['pos' 'n\\a' 'neg']
TEST: Total number of rows:  10000 , Unique labels: ['neg' 'pos']
TRAIN: Total number of rows:  43145 , Unique labels: ['pos' 'neg']
TEST: Total number of rows:  10000 , Unique labels: ['neg' 'pos']


In [7]:
new_df_train.head()

,lang,review,label
0,de,Einen Ausweg gibt es vielleicht für Daru (Vigg...,"[0, 1]"
2,pl,Bardzo lubię kino niemieckie. Jest fenomenalne...,"[0, 1]"
3,pl,Całkiem niedawno trafiłem na nie tak świeżą pr...,"[0, 1]"
4,cs,"V roce 1963, tedy v době, kdy se začalo mimo j...","[0, 1]"
5,pl,"Dzieciaki - mogłoby się wydawać, że nie mają n...","[0, 1]"


In [8]:
new_df_test.head()

,lang,review,label
0,pl,"Bardzo intensywnie i długo zastanawiałem się, ...","[1, 0]"
1,sk,"Režisér Richard Kelly, tvorca skvelého Donnieh...","[0, 1]"
2,sk,Stáva sa to pomerne často. Príprava filmu sa n...,"[0, 1]"
3,sk,Tretí diel rebootovaného Star Treku od J. J. A...,"[1, 0]"
4,de,"„Spieglein, Spieglein an der Wand…“ heißt es i...","[1, 0]"


In [9]:
new_df_train['review'].apply(lambda x: len(str(x).split())).max()

5703

In [18]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 3
LEARNING_RATE = 1e-05
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

In [11]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
#         self.lang = self.data["lang"]
        self.review = self.data["review"]
        self.targets = self.data["label"]
        self.max_len = max_len

    def __len__(self):
        return len(self.review)

    def __getitem__(self, index):
        review = str(self.review[index])
        review = " ".join(review.split())
#         lang = str(self.lang[index])

        inputs = self.tokenizer(
            review, 
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation='only_first', 
            return_overflowing_tokens=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [12]:
# Creating the dataset and dataloader for the neural network

train_dataset=new_df_train.sample(frac=1,random_state=200).reset_index(drop=True)
test_dataset=new_df_test.sample(frac=1,random_state=200).reset_index(drop=True)


# print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

TRAIN Dataset: (43145, 3)
TEST Dataset: (10000, 3)


In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [14]:
class XLNetClass(torch.nn.Module):
    def __init__(self):
        super(XLNetClass, self).__init__()
        self.l1 = XLNetModel.from_pretrained("xlnet-base-cased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output
    
model = XLNetClass()
model.to(device)

XLNetClass(
  (l1): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pre_classifier): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
)

In [15]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [17]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [20]:
loss_tracker = []
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%1000==0:
            loss_tracker.append(loss)
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
'''accuracy_tracker = []
f1_micro_tracker = []
f1_macro_tracker = []
for epoch in range(EPOCHS):
    print("Epoch ",epoch)
    train(epoch)
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    accuracy_tracker.append(accuracy)
    f1_micro_tracker.append(f1_score_micro)
    f1_macro_tracker.append(f1_score_macro)
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")'''

In [ ]:
'''np.savetxt('multilingual_review_xlnet_loss_tracker.txt',loss_tracker,delimiter=',')
np.savetxt('multilingual_review_xlnet_accuracy_tracker.txt',accuracy_tracker,delimiter=',')
np.savetxt('multilingual_review_xlnet_f1_micro_tracker.txt',f1_micro_tracker,delimiter=',')
np.savetxt('multilingual_review_xlnet_f1_macro_tracker.txt',f1_macro_tracker,delimiter=',')

Pkl_Filename = "multingual_review_xlnet.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)'''

In [21]:
# ... (your existing code)

accuracy_tracker = []
f1_micro_tracker = []
f1_macro_tracker = []
precision_tracker = []
recall_tracker = []
log_loss_tracker = []
hamming_loss_tracker = []

for epoch in range(EPOCHS):
    print("Epoch ",epoch)
    train(epoch)
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5

    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    
    # Calculate precision, recall
    precision = metrics.precision_score(targets, outputs, average='micro')
    recall = metrics.recall_score(targets, outputs, average='micro')

    # Calculate log loss
    log_loss_val = metrics.log_loss(targets, outputs)

    # Calculate hamming loss
    hamming_loss_val = metrics.hamming_loss(targets, outputs)

    accuracy_tracker.append(accuracy)
    f1_micro_tracker.append(f1_score_micro)
    f1_macro_tracker.append(f1_score_macro)
    precision_tracker.append(precision)
    recall_tracker.append(recall)
    log_loss_tracker.append(log_loss_val)
    hamming_loss_tracker.append(hamming_loss_val)

    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Log Loss = {log_loss_val}")
    print(f"Hamming Loss = {hamming_loss_val}")



Epoch  0
Epoch: 0, Loss:  0.8704323768615723
Epoch: 0, Loss:  0.40374475717544556
Epoch: 0, Loss:  0.5173176527023315
Epoch: 0, Loss:  0.3137751519680023
Epoch: 0, Loss:  0.6635217666625977
Epoch: 0, Loss:  0.2858777642250061
Accuracy Score = 0.5003
F1 Score (Micro) = 0.5004501350405122
F1 Score (Macro) = 0.3342883531733652
Precision = 0.5006003602161296
Recall = 0.5003
Log Loss = 17.989803294816703
Hamming Loss = 0.4994
Epoch  1
Epoch: 1, Loss:  0.25238949060440063
Epoch: 1, Loss:  0.5585943460464478
Epoch: 1, Loss:  0.30866703391075134
Epoch: 1, Loss:  0.15207642316818237
Epoch: 1, Loss:  0.8928340673446655
Epoch: 1, Loss:  0.6097288131713867
Accuracy Score = 0.574
F1 Score (Micro) = 0.5764305722288916
F1 Score (Macro) = 0.5191473515583979
Precision = 0.5766613290632506
Recall = 0.5762
Log Loss = 15.170773711479406
Hamming Loss = 0.4234
Epoch  2
Epoch: 2, Loss:  0.6176093816757202
Epoch: 2, Loss:  0.2725371718406677
Epoch: 2, Loss:  0.4001775085926056
Epoch: 2, Loss:  0.2752133309841

In [24]:
# Calculate final scores after all epochs
final_accuracy = np.mean(accuracy_tracker)
final_f1_micro = np.mean(f1_micro_tracker)
final_f1_macro = np.mean(f1_macro_tracker)
final_precision = np.mean(precision_tracker)
final_recall = np.mean(recall_tracker)
final_log_loss = np.mean(log_loss_tracker)
final_hamming_loss = np.mean(hamming_loss_tracker)

# Print final scores
print("Final Accuracy Score = {:.4f}".format(final_accuracy))
print("Final F1 Score (Micro) = {:.4f}".format(final_f1_micro))
print("Final F1 Score (Macro) = {:.4f}".format(final_f1_macro))
print("Final Precision = {:.4f}".format(final_precision))
print("Final Recall = {:.4f}".format(final_recall))
print("Final Log Loss = {:.4f}".format(final_log_loss))
print("Final Hamming Loss = {:.4f}".format(final_hamming_loss))

# Save metrics to files
np.savetxt('multilingual_review_xlnet_precision_tracker.txt', precision_tracker, delimiter=',')
np.savetxt('multilingual_review_xlnet_recall_tracker.txt', recall_tracker, delimiter=',')
np.savetxt('multilingual_review_xlnet_log_loss_tracker.txt', log_loss_tracker, delimiter=',')
np.savetxt('multilingual_review_xlnet_hamming_loss_tracker.txt', hamming_loss_tracker, delimiter=',')


Final Accuracy Score = 0.5468
Final F1 Score (Micro) = 0.5483
Final F1 Score (Macro) = 0.4459
Final Precision = 0.5490
Final Recall = 0.5476
Final Log Loss = 16.1901
Final Hamming Loss = 0.4512
